# 1) CNN Training

Training fresh CNN model using provided training and validation images. Information of CNN model architecture and training and validation images can be found at: https://github.com/keiserlab/plaquebox-paper.

You can train a new model all together, just specify the locations

In [1]:
import time, os
import torch
torch.manual_seed(123456789)
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
from torch.utils.data import Dataset
from PIL import Image
from os.path import join as oj

In [4]:
# Global parameters
DATA_DIR = '/mnt/Data/'
BATCH_SIZE = 48
NUM_WORKERS = 12
SAVE_DIR = '../models/'
SAVE_NAME = 'new_model_params.pkl'
CSV_PATH = {
    'train': '../CSVs/train_oversampled.csv',
    'dev': '../CSVs/dev_oversampled.csv'
}
IMAGE_CLASSES = ['cored','diffuse','CAA'] 

In [ ]:
# run training
DATA_DIR = oj(DATA_DIR, 'Tiles/train_and_val/')

class MultilabelDataset(Dataset):
    def __init__(self, csv_path, img_path, transform=None):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        self.data_info = pd.read_csv(csv_path)
        self.img_path = img_path
        self.transform = transform
        c=torch.Tensor(self.data_info.loc[:,'cored'])
        d=torch.Tensor(self.data_info.loc[:,'diffuse'])
        a=torch.Tensor(self.data_info.loc[:,'CAA'])
        c=c.view(c.shape[0],1)
        d=d.view(d.shape[0],1)
        a=a.view(a.shape[0],1)
        self.raw_labels = torch.cat([c,d,a], dim=1)
        self.labels = (torch.cat([c,d,a], dim=1)>0.99).type(torch.FloatTensor)

    def __getitem__(self, index):
        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.labels[index]
        raw_label = self.raw_labels[index]
        # Get image name from the pandas df
        single_image_name = str(self.data_info.loc[index,'imagename'])
        # Open image *** JC - I made changes to this part since NEGATIVE_DIR no longer exist ***
        img_as_img = Image.open(self.img_path + single_image_name)
        # Transform image to tensor
        if self.transform is not None:
            img_as_img = self.transform(img_as_img)
        # Return image and the label
        return (img_as_img, single_image_label, raw_label, single_image_name)

    def __len__(self):
        return len(self.data_info.index)
    
    
def imshow(inp, norm, title=None):
    """Imshow for Tensor."""
    try:
        inp = inp.numpy().transpose((1, 2, 0))
    except:
        inp = inp.transpose((1, 2, 0))
    mean = norm['mean']
    std = norm['std']
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.subplots()
    ax.imshow(inp)
    
    if title is not None:
        ax.set_title(title)
    plt.pause(0.001)
    
    
def plot_loss(model):
    fig = plt.figure()
    fig.subplots_adjust(left=2, right = 3, wspace=0.3, bottom = 2, top = 3)
    ax = plt.axes()

    ax.plot(model.train_loss_curve, label='train')
    ax.plot(model.dev_loss_curve, label='dev')
    ax.set_xlabel('epoch')
    ax.set_ylabel('loss')
    ax.legend()
    
    
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, gpu_id=None):
    since = time.time()

    best_loss = 10000.0
    best_model = copy.deepcopy(model)

    for epoch in range(num_epochs):
        epoch_time = time.time()

        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'dev']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  
            else:
                model.train(False)  

            running_loss = 0.0
            running_corrects = torch.zeros(len(IMAGE_CLASSES))

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels, raw_labels, names = data
                # wrap them in Variable
                if use_gpu:
                    if phase == 'train':
                        inputs = Variable(inputs.cuda(), requires_grad=True)
                    else:
                        inputs = Variable(inputs.cuda(), volatile=True)
                    labels = Variable(labels.cuda(), volatile=True)
                else:
                    if phase == 'train':
                        inputs =  Variable(inputs, requires_grad=True)
                    else:
                        inputs =  Variable(inputs, volatile=True)
                    labels = Variable(labels, volatile=True)

                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                outputs = model(inputs)
                if use_gpu:
                    predictions = (F.sigmoid(outputs)>0.5).type(torch.cuda.FloatTensor)
                else:
                    predictions = (F.sigmoid(outputs)>0.5).type(torch.FloatTensor)

                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(predictions==labels, 0).data.type(torch.FloatTensor)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            if phase == 'train':
                model.module.train_loss_curve.append(epoch_loss)
            else:
                model.module.dev_loss_curve.append(epoch_loss)

            if phase == 'dev' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model = copy.deepcopy(model)
                print('best loss: ', epoch_loss)

            print('{} Loss: {:.4f}\n Cored: {:.4f} Diffuse: {:.4f} CAA: {:.4f}'.format(
                phase, epoch_loss, epoch_acc[0], epoch_acc[1], epoch_acc[2]))

        epoch_end = time.time() - epoch_time
        print('train, Epoch time {:.0f}m {:.0f}s'.format(
                epoch_end // 60, epoch_end % 60))
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    return best_model


def dev_model(model, criterion, phase='dev'):
    phase = phase
    since = time.time()
    
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=False, num_workers=num_workers)
              for x in [phase]}

    model.train(False) 

    running_loss = 0.0
    running_corrects = torch.zeros(len(IMAGE_CLASSES))
    running_preds = torch.Tensor(0)          # confidence score
    running_predictions = torch.Tensor(0)    # classification prediction
    running_labels = torch.Tensor(0)
    running_raw_labels = torch.Tensor(0)

    # Iterate over data.
    for data in dataloaders[phase]:
        # get the inputs
        inputs, labels, raw_labels, names = data
        running_labels = torch.cat([running_labels, labels])
        running_raw_labels = torch.cat([running_raw_labels, raw_labels])

        # wrap them in Variable
        if use_gpu:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        # forward
        outputs = model(inputs)
        preds = F.sigmoid(outputs) #posibility for each class
        if use_gpu:
            predictions = (preds>0.5).type(torch.cuda.FloatTensor)
        else:
            predictions = (preds>0.5).type(torch.FloatTensor)
        
        loss = criterion(outputs, labels)

        preds = preds.data.cpu()
        predictions = predictions.data.cpu()
        labels = labels.data.cpu()

        # statistics
        running_loss += loss.data[0]
        running_corrects += torch.sum(predictions==labels, 0).type(torch.FloatTensor)
        running_preds = torch.cat([running_preds, preds])
        running_predictions = torch.cat([running_predictions, predictions])


    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects / dataset_sizes[phase]

    print('{} Loss: {:.4f}\n Cored: {:.4f} Diffuse: {:.4f} CAA: {:.4f}'.format(
                phase, epoch_loss, epoch_acc[0], epoch_acc[1], epoch_acc[2]))

    print()

    time_elapsed = time.time() - since
    print('Prediction complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return epoch_acc, running_preds, running_predictions, running_labels


class Net(nn.Module):

    def __init__(self, fc_nodes=512, num_classes=3, dropout=0.5):
        super(Net, self).__init__()
        
        self.drop = 0.2
     
        self.features = nn.Sequential(nn.Conv2d(3, 64, 3, padding=1),
                                      nn.Dropout2d(self.drop),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      
                                      nn.Conv2d(64, 64, 3, padding=1),
                                      nn.Dropout2d(self.drop),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      
                                      nn.Conv2d(64, 128, 3, padding=1),
                                      nn.Dropout2d(self.drop),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      
                                      nn.Conv2d(128, 256, 3, padding=1),
                                      nn.Dropout2d(self.drop),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      
                                      nn.Conv2d(256, 256, 3, padding=1),
                                      nn.Dropout2d(self.drop),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      
                                      nn.Conv2d(256, 512, 3, padding=1),
                                      nn.Dropout2d(self.drop),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=2, stride=2),)
        
        self.classifier = nn.Sequential(nn.Linear(512 * 4 * 4, fc_nodes),
                                        nn.ReLU(True),
                                        nn.Dropout(p=dropout),
                                        nn.Linear(fc_nodes, 100),
                                        nn.ReLU(True),
                                        nn.Dropout(p=dropout),
                                        nn.Linear(100, num_classes))
        
        self.train_loss_curve = []
        self.dev_loss_curve = []

    def forward(self, x):
 
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x
    

# create save directory
os.makedirs(SAVE_DIR, exist_ok=True)

# load color normalization info
norm = np.load('../modules/normalization.npy').item()

# create data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(180),
        transforms.ColorJitter(brightness=0.1, contrast=0.2,saturation=0.2, hue=0.02),
        transforms.RandomAffine(0, translate=(0.05,0.05), scale=(0.9,1.1), shear=10),
        transforms.ToTensor(),
        transforms.Normalize(norm['mean'], norm['std'])
    ]),
    'dev': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(norm['mean'], norm['std'])
    ]),
}

image_datasets = {x: MultilabelDataset(CSV_PATH[x], DATA_DIR, data_transforms[x])
              for x in ['train', 'dev']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                         shuffle=True, num_workers=NUM_WORKERS)
              for x in ['dev', 'train']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'dev']}
print("{} training images, {} validation images".format(dataset_sizes["train"], dataset_sizes["dev"]))

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("GPU found, will use it during training")
else:
    print("No GPU found, training on CPU")

# get a batch of training data, make into grid from batch size and show
inputs, labels, raw_labels, names = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs)
imshow(out, norm)

weight = torch.FloatTensor([1,1,1])
model = Net()

if use_gpu:
    weight = weight.cuda()
    # ||||||||||||||||||| if you have multiple GPUs - then set device_ids to [0, 1, 2, ...] ||||||||||||||| #
    model = nn.DataParallel(model, device_ids=[0, 1])
    model = model.cuda()

criterion = nn.MultiLabelSoftMarginLoss(weight=weight, size_average=False)
optimizer = optim.Adam(model.parameters(), lr=0.00008, weight_decay=0.008)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.4)

# train the model
best_model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=60)

# save the model
if os.path.isfile(oj(SAVE_DIR, SAVE_NAME)):
    print('File {} already exists - not saving new model'.format(oj(SAVE_DIR, SAVE_NAME)))
else:
    torch.save(best_model, os.path.join(SAVE_DIR, SAVE_NAME))

# plot the loss during training
plot_loss(model.module)
